In [ ]:
import scipy.integrate as sp1
import scipy.interpolate as sp2
import scipy.misc as sp3

import pandas as pd
import pickle as pkl
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import numba as nb
import os
from pathlib import Path
%matplotlib inline

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 100

In [ ]:
from matplotlib import rcParams
rcParams['figure.figsize'] = 8,8

In [ ]:
sns.set_context("paper")
sns.set(font='serif')
sns.set_style("white", {
        "font.family": "serif",
        "font.serif": ["Times", "Palatino", "serif"]})

pardir=Path(os.getcwd()).parent.parent
datadir=str(pardir) +'/Data'

mycmap=sns.cubehelix_palette(n_colors=50, start=1, rot=4, light=0.7, dark=0)

In [ ]:
def create4(reflect3,reflect4, w, fc, right, sigma, deriv, g): 
    for i in range(0,100*fc+1):
        for j in range(0,i+right-100*fc):
            der=deriv[int(round(j*(1-w)/fc))]
            score=sigma[int(round(j*(1-w)))]
            bid=(score-(1-w)*i)/w
            if bid<100*fc+1 and bid>-1:
                reflect4[i,j]=g(i,bid)*der

def gg(x,y,g):
    return g(x,y)[0,0]

def create5(reflect4,reflect5, bounds1,fc): 
    for i in range(0,100*fc+1):
        for j in range(i,bounds1[i]):
            reflect5[i,j]=reflect4[i,bounds1[i]]
            
@nb.autojit(nopython=True)
def create1(f,Fp1,fc,right,w):
    for i in range(0,100*fc+1):
        for j in range(0,right):
            Fp1[i,j]=(1-w)*np.sum(f[:i+1,j])/fc
            
@nb.autojit(nopython=True)
def create2(f,Fp2,fc,right,w): 
    for i in range(0,100*fc+1):
        for j in range(0,right):
            Fp2[i,j]=(1-w)*np.sum(f[i,:j+1])/fc
            
@nb.autojit(nopython=True)
def create0(Fp2,F,fc,right,w):
    for i in range(0,100*fc+1):
        for j in range(0,right):
            F[i,j]=(1-w)*np.sum(Fp2[:i+1,j])/fc
            
Z=1

@nb.autojit(nopython=True)
def create3fake(F,Fp1,Fp2,rhs,fc,right,w): 
    for i in range(0,100*fc+1):
        for j in range(i,right):
            if (j-i)!=0:
                den=-Fp2[i,j]+fc*Z*F[i,j]/((j-i)*(1-w))
                if den==0:
                    print(i,j)
                else:
                    my=Fp1[i,j]/den
                    if my>0:
                        rhs[i,j]=1
            rhs[0,0]=1/2
            rhs[0,1]=1/2
                            
@nb.autojit(nopython=True)
def create3(F,Fp1,Fp2,rhs,fc,right,w): 
    for i in range(0,100*fc+1):
        for j in range(i,right):
            if (j-i)!=0:
                den=-Fp2[i,j]+fc*Z*F[i,j]/((j-i)*(1-w))
                if den==0:
                    pass#print(i,j)
                else:
                    my=Fp1[i,j]/den
                    rhs[i,j]=my
            rhs[0,0]=1/2
            rhs[0,1]=1/2
    
def create6(reflect5,reflect6,fc,ww,h): 
    for i in range(0,100*fc+1):
        for j in range(0,100*fc+1):
            reflect6[i,j]=h(i,i+j*ww) 

@nb.jit(nopython=True)
def K(band):
    return np.array([np.power(1-np.power((band-x)/band,2),3) for x in range(1,band+1)]
                    +[np.power(1-np.power(x/band,2),3) for x in range(1,band)], dtype=np.float64)*35/32

@nb.jit(nopython=True)
def smooth(reflect, reflect2, fc):
    myfc=5*fc
    Kband_ps=K(myfc)
    Kband_qs=K(myfc)
    bound1=100*fc+1
    bound2=100*fc+1
    for j in range(0,bound1):
        bot=max(0,j-myfc)
        top=min(j+myfc-1,100*fc+1)
        for i in range(0,bound2):
            area=0            
            left=max(i-myfc,0)
            right=min(i+myfc-1, 100*fc+1)
            for kj in range(bot,top):
                mykj=Kband_qs[kj-j+myfc]
                for ki in range(left,right):
                    myki=Kband_ps[ki-i+myfc]
                    my=myki*mykj
                    reflect2[j,i]+=reflect[kj,ki]*my
                    area+=my
            reflect2[j,i]=reflect2[j,i]/area
    return 0

In [ ]:
for fc in [8]:
    print('fc=',fc)
    for industry in ["leg", "mil", "sci", "edu", "tec"]:
        print('industry=',industry)
        for weight in ['55','80']:
            print('weight=',weight)
            if industry != 'uni':
                mydir=datadir+"/derive/"+industry+weight+"/"

                [PART,DATA]=pkl.load(open(datadir+'/derive/'+industry+weight+"j.pkl", "rb"))
                sigma, haz=pkl.load(open(mydir+industry+weight+"j_strategy3_"+str(fc)+".pkl", "rb"))
                reflect3=pkl.load(open(mydir+industry+weight+"j_reflect3_"+str(fc)+".pkl", "rb"))
                xx=np.array(list(range(0,100*fc+1)))
                yy=np.array(list(range(0,100*fc+1)))

                strat = sp2.interp1d(np.divide(range(0,100*fc+1),fc),np.divide(sigma,fc), kind='quadratic')

                def strategy(x):
                    return strat(max(0,min(100,x)))

                arr=np.array(range(0,100*fc+1))
                derivative = sp2.interp1d(arr,[sp3.derivative(strategy,min(max(fc/4,i),100-fc/4),dx=fc) for i in arr], kind='quadratic')

                deriv=np.array([derivative(i/fc) for i in range(0,100*fc+1)])
                
                g = sp2.RectBivariateSpline(yy, xx, reflect3, kx=2, ky=2)

                w=int(weight)/100
                ww=w/(1-w)
                right=int(100*fc/(1-w))+1

                reflect4=np.zeros((100*fc+1,right),dtype=float)

                create4(reflect3,reflect4,w,fc,right,sigma,deriv,g)
                reflect4=fc*fc*reflect4/np.sum(reflect4)

                #print(gg(9,9,g),reflect3[9,9])

                #this is the first mass in the x axis
                bounds1=np.zeros(100*fc+1, dtype=int)
                for i in range(0,100*fc+1):
                    j=0
                    while j<right:
                        if reflect4[i,j]>0:
                            bounds1[i]=j
                            j=right
                        j+=1

                #this is the last mass in the y axis
                bounds2=np.zeros(right, dtype=int)
                for j in range(0,right):
                    i=100*fc
                    while i>-1:
                        if reflect4[i,j]>0:
                            bounds2[j]=i
                            i=-1
                        i-=1
                reflect5=reflect4

                create5(reflect4,reflect5,bounds1,fc)
                reflect5=(fc/(1-w))*(fc/(1-w))*reflect5/np.sum(reflect5)

                ########################
                ########################
                print('saving...')
                ax=sns.heatmap(reflect5, cmap=mycmap, cbar=0, square=1)
                ax.invert_yaxis()

                #plt.plot(range(0,right),[bounds2[i] for i in range(0,right)],'s', markersize=1, color= 'red')
                plt.plot([bounds1[i] for i in range(0,100*fc+1)],range(0,100*fc+1),'ro', markersize=1, color= 'orange')

                plt.plot(range(0,int(100*fc)),np.array(range(0,int(100*fc))), 'ro', markersize=1, color= 'b')
                plt.plot(100*fc*ww+np.array(range(0,int(100*fc))),np.array(range(0,int(100*fc))), 'ro', markersize=1, color= 'b')

                plt.tight_layout()
                #plt.savefig(datadir+"/derive/img/"+industry+weight+'_inter.png', bbox_inches='tight')
                
                plt.clf()
                ##########################
                ############################
                
                rhs=np.zeros((100*fc+1,right),dtype=float)
                rhsfake=np.zeros((100*fc+1,right),dtype=float)
                F=np.zeros((100*fc+1,right),dtype=float)
                Fp1=np.zeros((100*fc+1,right),dtype=float)
                Fp2=np.zeros((100*fc+1,right),dtype=float)

                create1(reflect5,Fp1,fc,right,w)
                create2(reflect5,Fp2,fc,right,w)
                create0(Fp2,F,fc,right,w)
                #print(np.max(F))
                create3fake(F,Fp1,Fp2,rhsfake,fc,right,w)
                create3(F,Fp1,Fp2,rhs,fc,right,w)

                gridz=rhs

                xx=np.array(list(range(0,right)))
                yy=np.array(list(range(0,100*fc+1)))

                g = sp2.RectBivariateSpline(yy, xx, gridz, kx=2, ky=2)

                def myfun(x,y):
                    return g(y,x)

                print('start ode...')

                myode=sp1.ode(myfun, jac=None)
                myode.set_initial_value(1*fc, 1*fc+1)
                myode.set_integrator('dopri5')

                dt=0.1
                x=[0]
                y=[0]
                while myode.successful() and y[-1] < 100*fc and x[-1]<right:
                    x=x+[myode.t+dt]
                    y=y+[myode.integrate(myode.t+dt)[0]]

                ########################
                ########################
                print('saving...')
                ax=sns.heatmap(rhsfake, cmap=mycmap, cbar=0, square=1)
                ax.invert_yaxis()
                plt.plot(np.array(x), np.array(y), 'ro', markersize=0.2, color= 'red')
                plt.ylim(0, 100*fc+1)
                plt.xlim(0, 200*fc+1)
                #plt.savefig(datadir+"/derive/img/"+industry+weight+'_funnel.png', bbox_inches='tight')
                plt.clf()
                ########################
                #########################


                reflect6=np.zeros((100*fc+1,100*fc+1),dtype=float)

                print('finish this one...')

                xx=np.array(list(range(0,right)))
                yy=np.array(list(range(0,100*fc+1)))

                h = sp2.RectBivariateSpline(yy, xx, reflect5, kx=2, ky=2)
                
                create6(reflect5,reflect6, fc, ww, h)
                
                reflect7=np.zeros(reflect6.shape)
                smooth(reflect6,reflect7,fc)
                
                reflect6=fc*fc*reflect7/np.sum(reflect7)

                ############################
                #############################
                print('saving...')
                ax=sns.heatmap(reflect6, cmap=mycmap, cbar=0, square=1)
                ax.invert_yaxis()
                #plt.plot((np.array(x)-np.array(y)), np.array(y), 'ro', markersize=0.2, color= 'red')
                #plt.savefig(datadir+"/derive/img/"+industry+weight+'_rmcq.png', bbox_inches='tight')
                plt.clf()
                #############################
                ############################

                addr=mydir+industry+weight+"j_rcmq_"+str(fc)+".pkl"
                pkl.dump(reflect6,open(addr, "wb"))
            elif weight=='55':
                mydir=datadir+"/derive/"+industry+weight+"/"
                reflect6=np.array([[1 for j in range(0,100*fc+1)] for i in range(0,100*fc+1)])
                addr=mydir+industry+weight+"j_rcmq_"+str(fc)+".pkl"
                pkl.dump(reflect6,open(addr, "wb"))
            elif weight=='80':
                mydir=datadir+"/derive/"+industry+weight+"/"
                reflect6=np.array([[i for j in range(0,100*fc+1)] for i in range(0,100*fc+1)])
                addr=mydir+industry+weight+"j_rcmq_"+str(fc)+".pkl"
                pkl.dump(reflect6,open(addr, "wb"))
                